In [ ]:
import numpy as np
import time 
import os  
import xarray as xr
import subprocess
import matplotlib.pyplot as plt
from matplotlib import colors
import cartopy.crs as ccrs
# import torch
# from torch import nn  

In [ ]:
## import self defined functions
from sys import path 
# insert at 1, 0 is the script path (or '' in REPL)
path.insert(1, '/tigress/cw55/local/python_lib')
from cg_funcs import global_mean_xarray 

In [ ]:
plt.rcParams['font.size'] = '6'

In [ ]:
file_path = '/scratch/gpfs/cw55/AM4/work/CTL2000s_testNNdev_stellarcpu_intelmpi_22_768PE/'
  
# ds = xr.open_dataset(file_path+'/HISTORY/20000101.atmos_8xdaily.tile2.nc')  
ds = xr.open_dataset(file_path+'/POSTP/20000101.atmos_8xdaily.nc') 
ds1 = ds.rename({'grid_yt':'lat','grid_xt':'lon'})

file_path = '/scratch/gpfs/cw55/AM4/work/CTL2000s_testNNdev_stellarcpu_intelmpi_22_768PE_base/'
# ds = xr.open_dataset(file_path+'/HISTORY/20000101.atmos_8xdaily.tile2.nc')  
ds = xr.open_dataset(file_path+'/POSTP/20000101.atmos_8xdaily.nc') 
ds2 = ds.rename({'grid_yt':'lat','grid_xt':'lon'})


In [ ]:
def plot_var12_diff(var1,level = np.arange(0,400,10)):
    fig, axs = plt.subplots(nrows=1,ncols=3,figsize=(8,2),dpi=200)
    axs = axs.flatten()
    
    for i in range(2):
        ax=axs[i]
        da = ds1[var1[i]][1:-1].mean('time')
        da.plot(ax=ax, levels=level, extend='both') 
        # ax.set_title(f'{var1[i]} GM: {da.mean().values:4.2f}')  
        ax.set_title(f'{var1[i]} GM: {global_mean_xarray(da).values:4.2f}')  
    ax=axs[2]
    level = np.arange(-20,20,2)
    da = (ds1[var1[0]].mean('time')-ds1[var1[1]].mean('time'))
    da.plot(ax=ax, levels=level, extend='both') 
    ax.set_title(f'diff GM: {da.mean().values:4.2f}')  
    for ax in axs:
        ax.set(xlabel=None,ylabel=None)
    fig.tight_layout()  
def plot_ds12_diff(ds_list,level = np.arange(0,300,10), dlevel = np.arange(-10,10.1,0.5)):
    fig, axs = plt.subplots(nrows=1,ncols=3,figsize=(8,2),dpi=200)
    axs = axs.flatten()
    for i in range(2):
        ax=axs[i]
        da = ds_list[i].mean('time').load()
        da.plot(ax=ax, levels=level, extend='both')  
        ax.set_title(f'GM: {global_mean_xarray(da).values:4.2f}') 
    ax=axs[2]
    da = (ds_list[1].mean('time')-ds_list[0].mean('time')).load()
    da.plot(ax=ax, levels=dlevel, extend='both') 
    ax.set_title(f'diff mean: {(da.mean()).values:4.2f}')  
    ax.set_title(f'diff GM: {global_mean_xarray(da).values:4.2f}') 
    for ax in axs:
        ax.set(xlabel=None,ylabel=None)
    fig.tight_layout()  
def plot_zonal_ta_diff(ds_list,level = np.arange(200,320,10), dlevel = np.arange(-1,1.1,0.1)):
    fig, axs = plt.subplots(nrows=1,ncols=3,figsize=(8,2),dpi=200)
    axs = axs.flatten()
    for i in range(2):
        ax=axs[i]
        da = ds_list[i].mean(['time','lon']).load()
        da.plot(ax=ax, levels=level, extend='both')  
        ax.set_title(f'mean: {(da.mean()).values:4.2f}')  
    ax=axs[2]
    da = (ds_list[1]-ds_list[0]).mean(['time','lon']).load()
    da.plot(ax=ax, levels=dlevel, extend='both') 
    ax.set_title(f'diff mean: {(da.mean()).values:4.2f}')  
    # ax.set_title(f'diff GM: {global_mean_xarray(da).values:4.2f}') 
    for ax in axs:
        ax.set(xlabel=None,ylabel=None)
        ax.invert_yaxis()
    fig.tight_layout()  

In [ ]:
var = 'temp'
ds_list = [ds1[var],ds2[var]]
plot_zonal_ta_diff(ds_list,level = np.arange(200,330,10), dlevel = np.arange(-1,1.1,0.05)) 

In [ ]:
var = 't_surf'
ds_list = [ds1[var] ,ds2[var] ]
plot_ds12_diff(ds_list,level = np.arange(200,330,5), dlevel = np.arange(-2,2.1,0.5)) 

In [ ]:
var = ['nn_olr_clr','olr_clr',]
plot_var12_diff(var)

var = ['nn_olr','olr',]
plot_var12_diff(var)

In [ ]:
var = ['nn_swup_toa_clr','swup_toa_clr',]
plot_var12_diff(var,level = np.arange(100,300,10),)
var = ['nn_swup_toa','swup_toa',]
plot_var12_diff(var,level = np.arange(100,300,10),)

In [ ]:
def plot_var12_diff2(var1,ds_sel):
    fig, axs = plt.subplots(nrows=1,ncols=3,figsize=(8,2),dpi=300)
    axs = axs.flatten()
    # level = np.arange(0,1400,10)
    for i in range(2):
        ax=axs[i]
        da = ds_sel[var1[i]]
        da.plot(ax=ax, extend='both') 
        # ax.set_title(f'{var1[i]} GM: {da.mean().values:4.2f}')
        ax.set_title(f'{var1[i]} GM: {global_mean_xarray(da).values:4.2f}')  
    ax=axs[2]
    # level = np.arange(-1000,1000,2)
    da = (ds_sel[var1[0]]-ds_sel[var1[1]])
    da.plot(ax=ax, extend='both') 
    ax.set_title(f'diff GM: {da.mean().values:4.2f}')  
    for ax in axs:
        ax.set(xlabel=None,ylabel=None)
    fig.tight_layout()   
# ds_sel = ds.mean('time')
ds_sel = ds1.isel(time=1) 
var = ['nn_swdn_toa','swdn_toa',]
plot_var12_diff2(var,ds_sel)  

In [ ]:
fig, axs = plt.subplots(nrows=1,ncols=3,figsize=(8,2),dpi=200)
axs = axs.flatten()
# level = np.arange(0,1400,10)
itime = 4
da_list = [ ds1['swup_toa_clr'][itime],
            ds1['nn_swup_toa_clr'][itime],] 
da_list.append(da_list[1]-da_list[0])
for i in range(3):
    ax=axs[i]
    da = da_list[i]
    da.plot(ax=ax, extend='both') 
    ax.set_title(f' GM: {da.mean().values:4.2f}')   
fig.tight_layout()  
fig.show()

# Diff in rsdt

In [ ]:
def plot_var12_diff2(var1,ds_sel,level1=np.arange(-500,1400,100),level2=np.arange(-500,500,100)):
    fig, axs = plt.subplots(nrows=1,ncols=3,figsize=(8,2),dpi=200)
    axs = axs.flatten() 
    for i in range(3): 
        ax=axs[i] 
        da = ds_sel[var1[0]]
        da.plot.contour(ax=ax, levels=np.arange(-500,1400,500)) 
    for i in range(2):
        ax=axs[i]
        da = ds_sel[var1[i]]
        da.plot(ax=ax, levels=level1, extend='both') 
        ax.set_title(f'{var1[i]} GM: {global_mean_xarray(da).values:4.2f}')  
    ax=axs[2]
    level = level2
    da = (ds_sel[var1[1]]-ds_sel[var1[0]])
    da.plot(ax=ax, levels=level2, extend='both') 
    ax.set_title(f'diff GM: {global_mean_xarray(da).values:4.2f}') 
    for ax in axs:
        ax.set(xlabel=None,ylabel=None)
    fig.tight_layout()   
# ds_sel = ds.mean('time')
ds_sel = ds1.isel(time=3) 
print(ds_sel.time.data)
var = ['nn_swdn_toa','swdn_toa',]
plot_var12_diff2(var,ds_sel)   

# 

# Check Error in Energe conservation

## lw

In [ ]:

hybrid_p_sigma_para = xr.open_dataset('/tigress/cw55/data/NNRTMC_dataset/AM4_pk_bk_202207.nc')
A_k = hybrid_p_sigma_para.ak.values
B_k = hybrid_p_sigma_para.bk.values 
def energy_error_lw_2d(rlut, rlus, rlds, tdt_lw, ps):
    C_p = 1004.64    # J/kg/K 
    g   = 9.8        # m/s^2  
    k = tdt_lw.shape[0]
    nlat = ps.shape[0] 
    P_lev = A_k[:,None,None] + np.matmul(B_k[:,None],ps.reshape(1,-1)).reshape(k+1,nlat,-1)
    dP = (P_lev[1:,:,:] - P_lev[:33,:,:])
    F_net = rlus - rlut - rlds 
    sum_Cphr_gdp = C_p/g * (tdt_lw*dP).sum(axis=0)
    E_err = F_net - sum_Cphr_gdp
    return E_err
def energy_error_sw_2d(rsut, rsdt, rsus, rsds, tdt_sw, ps):
    C_p = 1004.64    # J/kg/K 
    g   = 9.8        # m/s^2  
    k = tdt_lw.shape[0]
    nlat = ps.shape[0] 
    P_lev = A_k[:,None,None] + np.matmul(B_k[:,None],ps.reshape(1,-1)).reshape(k+1,nlat,-1)
    dP = (P_lev[1:,:,:] - P_lev[:33,:,:])
    F_net = rsdt - rsut + rsus - rsds 
    sum_Cphr_gdp = C_p/g * (tdt_sw*dP).sum(axis=0)
    E_err = F_net - sum_Cphr_gdp
    return E_err

In [ ]:
ds_sel = ds1.isel(time=[0,1,2,3,4,5,6]).mean('time')
ds_sel = ds1.isel(time=[4]).mean('time')
rlut = ds_sel['olr'].values
rlus = ds_sel['lwup_sfc'].values
rlds = ds_sel['lwdn_sfc'].values
tdt_lw = ds_sel['tdt_lw'].values
ps   = ds_sel['nn_plevel'].values[-1,:,:]
eng_err = energy_error_lw_2d(rlut, rlus, rlds, tdt_lw, ps)
fig, axs = plt.subplots(nrows=1,ncols=2,figsize=(6,2),dpi=200)
axs = axs.flatten()
h = axs[0].contourf(eng_err,levels = np.arange(-2,2.01,0.1))
# plt.colorbar(ax =axs[0])
fig.colorbar(h, ax=axs[0])
axs[1].hist(eng_err.flatten(),bins=30)
axs[1].set_yscale('log')
fig.show()

In [ ]:
ds_sel = ds2.isel(time=[0,1]).mean('time')
ds_sel = ds2.isel(time=[0]).mean('time')
skycond = '_clr'
skycond = ''
rlut = ds_sel['nn_olr'+skycond].values
rlus = ds_sel['nn_lwup_sfc'].values
rlds = ds_sel['nn_lwdn_sfc'+skycond].values
tdt_lw = ds_sel['nn_tdt_lw'+skycond].values
ps   = ds_sel['nn_plevel'].values[-1,:,:]
eng_err = energy_error_lw_2d(rlut, rlus, rlds, tdt_lw, ps)
fig, axs = plt.subplots(nrows=1,ncols=2,figsize=(6,2),dpi=200)
axs = axs.flatten()
h = axs[0].contourf(eng_err )
fig.colorbar(h, ax=axs[0])
axs[1].hist(eng_err.flatten(),bins=30)
axs[1].set_yscale('log')
ind = np.argwhere(rlus.flatten()>-100)
axs[1].hist(eng_err.flatten()[ind],bins=30)
axs[1].set_yscale('log')
fig.show()

In [ ]:
ds_sel = ds2.isel(time=[0,1]).mean('time')
ds_sel = ds2.isel(time=[0]).mean('time') 
rlus = ds_sel['nn_lwup_sfc'].values  
fig, axs = plt.subplots(nrows=1,ncols=2,figsize=(6,2),dpi=200)
axs = axs.flatten()
h = axs[0].contourf(rlus ) 
fig.colorbar(h, ax=axs[0])
axs[1].hist(rlus.flatten(),bins=30)
axs[1].set_yscale('log')
ind = np.argwhere(rlus.flatten()>-100)
axs[1].hist(rlus.flatten()[ind],bins=30)
axs[1].set_yscale('log')
fig.show()

##  sw

In [ ]:
ds_sel = ds.isel(time=[0,1]).mean('time')
rsut = ds_sel['swup_toa'].values
rsdt = ds_sel['swdn_toa'].values
rsus = ds_sel['swup_sfc'].values
rsds = ds_sel['swdn_sfc'].values
tdt_sw = ds_sel['tdt_sw'].values
ps   = ds_sel['nn_plevel'].values[-1,:,:]
eng_err = energy_error_sw_2d(rsut, rsdt, rsus, rsds, tdt_sw, ps)
fig, axs = plt.subplots(nrows=1,ncols=2,figsize=(6,2),dpi=200)
axs = axs.flatten()
h = axs[0].contourf(eng_err,levels = np.arange(-2,2.01,0.1))
# plt.colorbar(ax =axs[0])
fig.colorbar(h, ax=axs[0])
axs[1].hist(eng_err.flatten(),bins=30)
axs[1].set_yscale('log')
fig.show()

In [ ]:
ds_sel = ds2.isel(time=[0,1]).mean('time')
rsut = ds_sel['nn_swup_toa'].values
rsdt = ds_sel['nn_swdn_toa'].values
rsus = ds_sel['nn_swup_sfc'].values
rsds = ds_sel['nn_swdn_sfc'].values
tdt_sw = ds_sel['nn_tdt_sw'].values
ps   = ds_sel['nn_plevel'].values[-1,:,:]
eng_err = energy_error_sw_2d(rsut, rsdt, rsus, rsds, tdt_sw, ps)
fig, axs = plt.subplots(nrows=1,ncols=2,figsize=(6,2),dpi=200)
axs = axs.flatten()
h = axs[0].contourf(eng_err,levels = np.arange(-2,2.01,0.1))
# plt.colorbar(ax =axs[0])
fig.colorbar(h, ax=axs[0])
axs[1].hist(eng_err.flatten(),bins=30)
axs[1].set_yscale('log')
fig.show()

In [ ]:
ds_sel = ds1

fig, axs = plt.subplots(nrows=1,ncols=2,figsize=(6,2),dpi=200)
axs = axs.flatten()
data = ds_sel['swdn_sfc'].values
axs[0].hist(data.flatten(),bins=30)
axs[0].set_yscale('log')
data = ds_sel['nn_swdn_sfc'].values
axs[1].hist(data.flatten(),bins=30)
axs[1].set_yscale('log')
fig.show()

In [ ]:
ds_sel = ds1

fig, axs = plt.subplots(nrows=1,ncols=2,figsize=(6,2),dpi=200)
axs = axs.flatten()
data = ds_sel['swdn_sfc'].values
axs[0].hist(data.flatten(),bins=30)
axs[0].set_yscale('log')
data = ds_sel['nn_swdn_sfc'].values - ds_sel['swdn_sfc'].values
axs[1].hist(data.flatten(),bins=30)
axs[1].set_yscale('log')
fig.show()

In [ ]:
ds_sel = ds1

fig, axs = plt.subplots(nrows=1,ncols=2,figsize=(6,2),dpi=200)
axs = axs.flatten()
data = ds_sel['swup_sfc'].values
axs[0].hist(data.flatten(),bins=30)
axs[0].set_yscale('log')
data = ds_sel['nn_swup_sfc'].values
axs[1].hist(data.flatten(),bins=30)
axs[1].set_yscale('log')
fig.show()

In [ ]:
ds_sel = ds1

fig, axs = plt.subplots(nrows=1,ncols=2,figsize=(6,2),dpi=200)
axs = axs.flatten()
data = ds_sel['swup_sfc'].values/ds_sel['swdn_sfc'].values
axs[0].hist(data.flatten(),bins=30)
axs[0].set_yscale('log')
data = ds_sel['nn_swup_sfc'].values/ds_sel['nn_swdn_sfc'].values
axs[1].hist(data.flatten(),bins=30)
axs[1].set_yscale('log')
fig.show()

In [ ]:
ds_sel = ds1

fig, axs = plt.subplots(nrows=1,ncols=2,figsize=(6,2),dpi=200)
axs = axs.flatten()
data = ds_sel['swdn_sfc'].values/ds_sel['swdn_toa'].values
axs[0].hist(data.flatten(),bins=30)
axs[0].set_yscale('log')
data = ds_sel['nn_swdn_sfc'].values/ds_sel['nn_swdn_toa'].values
axs[1].hist(data.flatten(),bins=30)
axs[1].set_yscale('log')
fig.show()

In [ ]:
ds_sel = ds1

fig, axs = plt.subplots(nrows=1,ncols=2,figsize=(6,2),dpi=200)
axs = axs.flatten()
data = ds_sel['swup_toa'].values/ds_sel['swdn_toa'].values
axs[0].hist(data.flatten(),bins=30)
axs[0].set_yscale('log')
data = ds_sel['nn_swup_toa'].values/ds_sel['nn_swdn_toa'].values
axs[1].hist(data.flatten(),bins=30)
axs[1].set_yscale('log')
fig.show()

In [ ]:
ds_sel = ds1

fig, axs = plt.subplots(nrows=1,ncols=2,figsize=(6,2),dpi=200)
axs = axs.flatten()
data = ds_sel['lwdn_sfc'].values/ds_sel['lwup_sfc'].values
axs[0].hist(data.flatten(),bins=30)
axs[0].set_yscale('log')
data = ds_sel['nn_lwdn_sfc'].values/ds_sel['nn_lwup_sfc'].values
axs[1].hist(data.flatten(),bins=30)
axs[1].set_yscale('log')
fig.show()

In [ ]:
ds_sel = ds1

fig, axs = plt.subplots(nrows=1,ncols=2,figsize=(6,2),dpi=200)
axs = axs.flatten()
data = ds_sel['olr'].values/ds_sel['lwup_sfc'].values
axs[0].hist(data.flatten(),bins=30)
axs[0].set_yscale('log')
data = ds_sel['nn_olr'].values/ds_sel['nn_lwup_sfc'].values
axs[1].hist(data.flatten(),bins=30)
axs[1].set_yscale('log')
fig.show()

In [ ]:
# ds_sel = ds1

# fig, axs = plt.subplots(nrows=1,ncols=2,figsize=(6,2),dpi=200)
# axs = axs.flatten()
# data = ds_sel['tdt_sw'].values*86400
# axs[0].hist(data.flatten(),bins=30)
# axs[0].set_yscale('log')
# data = ds_sel['tdt_lw'].values*86400
# axs[1].hist(data.flatten(),bins=30)
# axs[1].set_yscale('log')
# fig.show()

In [ ]:
# ds_sel = ds1

# fig, axs = plt.subplots(nrows=1,ncols=2,figsize=(6,2),dpi=200)
# axs = axs.flatten()
# data = ds_sel['tdt_lw'].values*86400
# axs[0].hist(data.flatten(),bins=30)
# axs[0].set_yscale('log')
# shape4d = ds_sel['tdt_lw'].shape
# a2, b2 = xr.broadcast(ds_sel['tdt_lw'], ds_sel['lwup_sfc'])
# data = a2.values/b2.values
# axs[1].hist(data.flatten(),bins=30)
# axs[1].set_yscale('log')
# fig.show()

In [ ]:
# ds_sel = ds1

# fig, axs = plt.subplots(nrows=1,ncols=2,figsize=(6,2),dpi=200)
# axs = axs.flatten()
# data = ds_sel['tdt_sw_clr'].values*86400
# axs[0].hist(data.flatten(),bins=30)
# axs[0].set_yscale('log')
# data = ds_sel['tdt_lw_clr'].values*86400
# axs[1].hist(data.flatten(),bins=30)
# axs[1].set_yscale('log')
# fig.show()

In [ ]:
# ds_sel = ds1
# for k in range(0,33,5):
#     fig, axs = plt.subplots(nrows=1,ncols=2,figsize=(6,2),dpi=200)
#     axs = axs.flatten()
#     data = ds_sel['nn_tdt_sw'][:,k,:,:].values*86400
#     axs[0].hist(data.flatten(),bins=30)
#     axs[0].set_yscale('log')
#     data = ds_sel['nn_tdt_lw'][:,k,:,:].values*86400
#     axs[1].hist(data.flatten(),bins=30) 
#     axs[1].set_yscale('log')
#     fig.show()

In [ ]:

# skycond = '_clr'
# skycond = ''
# for fi in range(1,7):
#     file_path = '/scratch/gpfs/cw55/AM4/work/CTL2000s_testNNdev_stellarcpu_intelmpi_22_768PE/'

#     file_path = '/scratch/gpfs/cw55/AM4/work/CTL2000s_testNNdev_stellarcpu_intelmpi_22_768PE_base/'

#     ds = xr.open_dataset(file_path+f'/HISTORY/20000101.atmos_8xdaily.tile{fi}.nc')  
#     # ds = xr.open_dataset(file_path+'/POSTP/20000101.atmos_8xdaily.nc') 

#     ds_sel = ds

#     fig, axs = plt.subplots(nrows=1,ncols=2,figsize=(6,2),dpi=100)
#     axs = axs.flatten()
#     data = ds_sel['nn_tdt_lw'+skycond].values
#     axs[0].hist(data.flatten()*86400,bins=30)
#     axs[0].set_yscale('log')
#     data = ds_sel['nn_tdt_lw'+skycond].values-ds_sel['tdt_lw'+skycond].values
#     axs[1].hist(data.flatten()*86400,bins=30)
#     axs[1].set_yscale('log')
#     fig.show() 
#     fig, axs = plt.subplots(nrows=1,ncols=2,figsize=(6,2),dpi=100)
#     axs = axs.flatten()
#     data = ds_sel['nn_tdt_sw'+skycond].values
#     axs[0].hist(data.flatten()*86400,bins=30)
#     axs[0].set_yscale('log')
#     data = ds_sel['nn_tdt_sw'+skycond].values-ds_sel['tdt_sw'+skycond].values
#     axs[1].hist(data.flatten()*86400,bins=30)
#     axs[1].set_yscale('log')
#     fig.show()

#     data = ds_sel['nn_tdt_sw'+skycond].values-ds_sel['tdt_sw'+skycond].values
#     print(np.max(np.abs(data*86400)))

#     data = ds_sel['nn_tdt_lw'+skycond].values-ds_sel['tdt_lw'+skycond].values
#     print(np.max(np.abs(data*86400)))

# nn ratio

In [ ]:
 
file_path = '/scratch/gpfs/cw55/AM4/work/CTL2000s_testNNdev_stellarcpu_intelmpi_22_768PE/' 
for fi in range(1,7):
    ds = xr.open_dataset(file_path+f'/HISTORY/20000101.atmos_8xdaily.tile{fi}.nc')  
    # ds = xr.open_dataset(file_path+'/POSTP/20000101.atmos_8xdaily.nc') 

    ds_sel = ds
    data = ds_sel['nn_lwup_sfc'].values
    tmp = np.where(data<50, 1, 0 )
    print(np.sum(tmp), data.size, f'{np.sum(tmp)/data.size*100:5.2f}%')

ds = xr.open_dataset(file_path+'/POSTP/20000101.atmos_8xdaily.nc') 
ds_sel = ds1
data = ds_sel['nn_lwup_sfc'].values
tmp = np.where(data<50, 1, 0 )
print(np.sum(tmp), data.size, f'{np.sum(tmp)/data.size*100:5.2f}%')

In [ ]:
 
file_path = '/scratch/gpfs/cw55/AM4/work/CTL2000s_testNNdev_stellarcpu_intelmpi_22_768PE_base/'
for fi in range(1,7):
    ds = xr.open_dataset(file_path+f'/HISTORY/20000101.atmos_8xdaily.tile{fi}.nc')  
    # ds = xr.open_dataset(file_path+'/POSTP/20000101.atmos_8xdaily.nc') 

    ds_sel = ds
    data = ds_sel['nn_lwup_sfc'].values
    tmp = np.where(data<50, 1, 0 )
    print(np.sum(tmp), data.size, f'{np.sum(tmp)/data.size*100:5.2f}%')

ds = xr.open_dataset(file_path+'/POSTP/20000101.atmos_8xdaily.nc') 
ds_sel = ds1
data = ds_sel['nn_lwup_sfc'].values
tmp = np.where(data<50, 1, 0 )
print(np.sum(tmp), data.size, f'{np.sum(tmp)/data.size*100:5.2f}%')